In [1]:
import sys
sys.path.append('/root/semi/VFPUMC02')
%load_ext autoreload
%autoreload 2

In [3]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import *
from semilearn import get_config, split_ssl_data, BasicDataset,get_data_loader, get_algorithm, get_net_builder,Trainer
from torchvision import transforms
from enums.HideRatio import HideRatio

In [ ]:
baseConfig = {
    'algorithm': 'fixmatch',
    'net': 'vit_tiny_patch2_32',
    'use_pretrain': True, 
    'pretrain_path': 'https://github.com/microsoft/Semi-supervised-learning/releases/download/v.0.0.0/vit_tiny_patch2_32_mlp_im_1k_32.pth',
    'include_lb_to_ulb': True,
    'batch_size': 8,
    'uratio': 2,
    'eval_batch_size':8,
    'data_dir': '/root/semi/Semi-supervised-learning/notebooks/data',
    'distributed': False,
    'gpu': 0,
    'num_classes': 2,
    'num_train_iter': 1, 
    'num_eval_iter': 1,   
    'num_log_iter': 50, 
    'epoch': 1, 
}

In [7]:
for algorithm in algorithms:
    for dataset in datasets:
        for member in HideRatio:
            baseConfig['algorithm'] = 'fixmatch'
            baseConfig['num_labels'] = nearest_even(len(dataset.df) * member.value)
            baseConfig['ulb_num_labels'] = nearest_even(len(dataset.df) * (1 - member.value))
            config = get_config(baseConfig)
            result = train_and_get_eval(dataset, config)
            """
                result:
                    {'acc': 0.625, 'precision': 0.3125, 'recall': 0.5, 'f1': 0.38461538461538464}
            """
我想把这段代码封装成一个方法，参数是algorithms(字符串列表)、datasets、HideRatio（枚举类）
返回：DataFrame，一级索引:algorithm，二级索引：dataset.baseFileName, 三级索引：member.name

SyntaxError: invalid syntax (3645429636.py, line 1)

In [ ]:
def train_and_get_eval(dataset, config):
    # create model and specify algorithm
    algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)
    data = expand_to_image_shape(normalize_columns(dataset.df.values))
    data = (data * 255).astype(np.uint8)
    target = dataset.y
    lb_data, lb_target, ulb_data, ulb_target = split_ssl_data(config, data, target, 2,
                                                          config.num_labels,ulb_num_labels=config.ulb_num_labels,
                                                          include_lb_to_ulb=config.include_lb_to_ulb)
    train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomCrop(32, padding=int(32 * 0.125), padding_mode='reflect'),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

    train_strong_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                                 transforms.RandomCrop(32, padding=int(32 * 0.125), padding_mode='reflect'),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

    eval_transform = transforms.Compose([transforms.Resize(32),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

    lb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=False)
    ulb_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, train_transform, is_ulb=True, strong_transform=train_strong_transform)
    eval_dataset = BasicDataset(config.algorithm, lb_data, lb_target, config.num_classes, eval_transform, is_ulb=False)

    # define data loaders
    train_lb_loader = get_data_loader(config, lb_dataset, config.batch_size)
    train_ulb_loader = get_data_loader(config, ulb_dataset, int(config.batch_size * config.uratio))
    eval_loader = get_data_loader(config, eval_dataset, config.eval_batch_size)

        # training and evaluation
    trainer = Trainer(config, algorithm)
    trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)
    return trainer.evaluate(eval_loader)

In [ ]:
result:
{'acc': 0.625, 'precision': 0.3125, 'recall': 0.5, 'f1': 0.38461538461538464}